In [1]:
import pandas as pd
import numpy as np

## 1. Cleaning Main Dataset

This dataset was sourced from Kaggle (https://www.kaggle.com/datasets/arnabchaki/data-science-salaries-2023), and describes data science-related job titles and salaries. 

In [2]:
df = pd.read_csv('data/ds_salaries.csv')
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


In [3]:
for col in df.columns:
    print(col, df[col].unique())

work_year [2023 2022 2020 2021]
experience_level ['SE' 'MI' 'EN' 'EX']
employment_type ['FT' 'CT' 'FL' 'PT']
job_title ['Principal Data Scientist' 'ML Engineer' 'Data Scientist'
 'Applied Scientist' 'Data Analyst' 'Data Modeler' 'Research Engineer'
 'Analytics Engineer' 'Business Intelligence Engineer'
 'Machine Learning Engineer' 'Data Strategist' 'Data Engineer'
 'Computer Vision Engineer' 'Data Quality Analyst'
 'Compliance Data Analyst' 'Data Architect'
 'Applied Machine Learning Engineer' 'AI Developer' 'Research Scientist'
 'Data Analytics Manager' 'Business Data Analyst' 'Applied Data Scientist'
 'Staff Data Analyst' 'ETL Engineer' 'Data DevOps Engineer' 'Head of Data'
 'Data Science Manager' 'Data Manager' 'Machine Learning Researcher'
 'Big Data Engineer' 'Data Specialist' 'Lead Data Analyst'
 'BI Data Engineer' 'Director of Data Science'
 'Machine Learning Scientist' 'MLOps Engineer' 'AI Scientist'
 'Autonomous Vehicle Technician' 'Applied Machine Learning Scientist'
 'Lead D

This dataset looks pretty clean, so we don't need to do any pre-cleaning before importing its data into our SQL program. Awesome!

However, we will need to add an 'id' column as a unique primary key column.

In [4]:
df.index.names = ['id']
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
id,,,,,,,,,,,
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


In [5]:
# exporting as CSV, so we can read the cleaned CSV into SQL
df.to_csv('data/ds_salaries_fixed.csv')

Finally, we see that three columns (employee_residence, company_location, salary_currency) have country/currency codes, and it would be nice to have more info on these codes. We will add these as secondary joining tables in our SQL program to get a more complete picture of the data.

## 2. Cleaning Secondary Dataset (ISO 3166 Codes)

We want more information on the country codes in the (employee_residence, company_location) columns of the main dataset. The main dataset metadata indicates that these columns use ISO 3166 codes, so I found a dataset for ISO 3166 codes (https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv). Now we need to examine and clean the data before importing it into our SQL program as a table. We will be joining our main dataset and this dataset on the country code columns.

In [6]:
iso3166 = pd.read_csv('data/iso_3166.csv')
iso3166.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


We see that the column headers have '-' between words. This is a non-standard format, and SQL will throw errors when we try to ingest this data as-is. We need to replace the '-' with an underscore.

In [7]:
iso3166.columns =  iso3166.columns.str.replace('-','_')
iso3166.head(2)

,name,alpha_2,alpha_3,country_code,iso_3166_2,region,sub_region,intermediate_region,region_code,sub_region_code,intermediate_region_code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN


Now let's check for data types, so that when creating the table in SQL, we specify the correct data type.

In [8]:
iso3166.dtypes

name                         object
alpha_2                      object
alpha_3                      object
country_code                  int64
iso_3166_2                   object
region                       object
sub_region                   object
intermediate_region          object
region_code                 float64
sub_region_code             float64
intermediate_region_code    float64
dtype: object

Examining the raw data a little more, we see that ('country_code','region_code','sub_region_code','intermediate_region_code') columns should be int rather than float. Let's convert those!

In [9]:
for col in ['country_code','region_code','sub_region_code','intermediate_region_code']:
    iso3166[col] = iso3166[col].fillna(0.0).astype(int)
iso3166.head()

,name,alpha_2,alpha_3,country_code,iso_3166_2,region,sub_region,intermediate_region,region_code,sub_region_code,intermediate_region_code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142,34,0
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150,154,0
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150,39,0
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2,15,0
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9,61,0


In [10]:
iso3166.iloc[153]

name                                   Namibia
alpha_2                                    NaN
alpha_3                                    NAM
country_code                               516
iso_3166_2                       ISO 3166-2:NA
region                                  Africa
sub_region                  Sub-Saharan Africa
intermediate_region            Southern Africa
region_code                                  2
sub_region_code                            202
intermediate_region_code                    18
Name: 153, dtype: object

In [11]:
iso3166['alpha_2'][153] = 'NA'

/var/folders/n9/3mjn__fd1xd66lc1zh9nlj0h0000gp/T/ipykernel_33206/2393023246.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iso3166['alpha_2'][153] = 'NA'


In [12]:
# final data type check
iso3166.dtypes

name                        object
alpha_2                     object
alpha_3                     object
country_code                 int64
iso_3166_2                  object
region                      object
sub_region                  object
intermediate_region         object
region_code                  int64
sub_region_code              int64
intermediate_region_code     int64
dtype: object

One more thing - we want to check if our column of interest (alpha_2) is unique or not. Since the column has country codes for each country, we shouldn't expect duplicates, but just to be sure, we will check.

In [13]:
len(iso3166) == len(iso3166['alpha_2'].unique())

True

In [14]:
# exporting as CSV, so we can read the cleaned CSV into SQL
iso3166.to_csv('data/iso_3166_fixed.csv')

## 3. Cleaning Secondary Dataset (ISO 4217 Codes)

We want more information on the country codes in the (salary_currency) column of the main dataset. The main dataset metadata indicates that these columns use ISO 4217 codes, so I found a dataset for ISO 4217 codes (https://datahub.io/core/currency-codes). Now we need to examine and clean the data before importing it into our SQL program as a table. We will be joining our main dataset and this dataset on the currency code columns.

In [15]:
iso4217 = pd.read_csv('data/iso_4217.csv')
iso4217.head()

,Entity,Currency,AlphabeticCode,NumericCode,MinorUnit,WithdrawalDate
0,AFGHANISTAN,Afghani,AFN,971.0,2,NaN
1,ÅLAND ISLANDS,Euro,EUR,978.0,2,NaN
2,ALBANIA,Lek,ALL,8.0,2,NaN
3,ALGERIA,Algerian Dinar,DZD,12.0,2,NaN
4,AMERICAN SAMOA,US Dollar,USD,840.0,2,NaN


In [16]:
for col in iso4217.columns:
    print(col, iso4217[col].unique())

Entity ['AFGHANISTAN' 'ÅLAND ISLANDS' 'ALBANIA' 'ALGERIA' 'AMERICAN SAMOA'
 'ANDORRA' 'ANGOLA' 'ANGUILLA' 'ANTARCTICA' 'ANTIGUA AND BARBUDA'
 'ARGENTINA' 'ARMENIA' 'ARUBA' 'AUSTRALIA' 'AUSTRIA' 'AZERBAIJAN'
 'BAHAMAS (THE)' 'BAHRAIN' 'BANGLADESH' 'BARBADOS' 'BELARUS' 'BELGIUM'
 'BELIZE' 'BENIN' 'BERMUDA' 'BHUTAN' 'BOLIVIA (PLURINATIONAL STATE OF)'
 'BONAIRE, SINT EUSTATIUS AND SABA' 'BOSNIA AND HERZEGOVINA' 'BOTSWANA'
 'BOUVET ISLAND' 'BRAZIL' 'BRITISH INDIAN OCEAN TERRITORY (THE)'
 'BRUNEI DARUSSALAM' 'BULGARIA' 'BURKINA FASO' 'BURUNDI' 'CABO VERDE'
 'CAMBODIA' 'CAMEROON' 'CANADA' 'CAYMAN ISLANDS (THE)'
 'CENTRAL AFRICAN REPUBLIC (THE)' 'CHAD' 'CHILE' 'CHINA'
 'CHRISTMAS ISLAND' 'COCOS (KEELING) ISLANDS (THE)' 'COLOMBIA'
 'COMOROS (THE)' 'CONGO (THE DEMOCRATIC REPUBLIC OF THE)' 'CONGO (THE)'
 'COOK ISLANDS (THE)' 'COSTA RICA' "CÔTE D'IVOIRE" 'CROATIA' 'CUBA'
 'CURAÇAO' 'CYPRUS' 'CZECHIA' 'DENMARK' 'DJIBOUTI' 'DOMINICA'
 'DOMINICAN REPUBLIC (THE)' 'ECUADOR' 'EGYPT' 'EL SALVADOR'
 'EQUA

Most of the columns and their data seem fine, but the column MinorUnit has strings of numbers, nan, and '-'. We will first replace the '-' with a 0, and then convert the data type to int.

In [17]:
iso4217['MinorUnit'] = iso4217['MinorUnit'].replace('-',0)

In [18]:
for col in ['NumericCode','MinorUnit']:
    iso4217[col] = iso4217[col].fillna(0.0).astype(int)
iso4217.head()

,Entity,Currency,AlphabeticCode,NumericCode,MinorUnit,WithdrawalDate
0,AFGHANISTAN,Afghani,AFN,971,2,NaN
1,ÅLAND ISLANDS,Euro,EUR,978,2,NaN
2,ALBANIA,Lek,ALL,8,2,NaN
3,ALGERIA,Algerian Dinar,DZD,12,2,NaN
4,AMERICAN SAMOA,US Dollar,USD,840,2,NaN


In [19]:
# doing final data type check
iso4217.dtypes

Entity            object
Currency          object
AlphabeticCode    object
NumericCode        int64
MinorUnit          int64
WithdrawalDate    object
dtype: object

In [20]:
len(iso4217) == len(iso4217['AlphabeticCode'].unique())

False

In [21]:
# exporting as CSV, so we can read the cleaned CSV into SQL
iso4217.to_csv('data/iso_4217_fixed.csv')